# Amazon E-commerce — Data Cleaning

**Goal:** convert raw sales report into a clean, analysis-ready dataset.


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("amazon_sales.csv", low_memory=False)
df.shape

(128975, 24)

## 1) Standardize columns & remove junk columns

In [4]:
df.columns = [c.strip() for c in df.columns]
df = df.loc[:, ~df.columns.str.contains(r'^Unnamed', case=False)]
df.columns.tolist()

['index',
 'Order ID',
 'Date',
 'Status',
 'Fulfilment',
 'Sales Channel',
 'ship-service-level',
 'Style',
 'SKU',
 'Category',
 'Size',
 'ASIN',
 'Courier Status',
 'Qty',
 'currency',
 'Amount',
 'ship-city',
 'ship-state',
 'ship-postal-code',
 'ship-country',
 'promotion-ids',
 'B2B',
 'fulfilled-by']

## 2) Rename columns (snake_case)

In [5]:
rename_map = {
    'Order ID':'order_id',
    'Date':'date',
    'Status':'status',
    'Fulfilment':'fulfilment',
    'Sales Channel ':'sales_channel',
    'Sales Channel':'sales_channel',
    'ship-service-level':'ship_service_level',
    'Style':'style',
    'SKU':'sku',
    'Category':'category',
    'Size':'size',
    'ASIN':'asin',
    'Courier Status':'courier_status',
    'Qty':'qty',
    'currency':'currency',
    'Amount':'amount',
    'ship-city':'ship_city',
    'ship-state':'ship_state',
    'ship-postal-code':'ship_postal_code',
    'ship-country':'ship_country',
    'promotion-ids':'promotion_ids',
    'B2B':'b2b',
    'fulfilled-by':'fulfilled_by'
}
df = df.rename(columns=rename_map)
df.columns.tolist()


['index',
 'order_id',
 'date',
 'status',
 'fulfilment',
 'sales_channel',
 'ship_service_level',
 'style',
 'sku',
 'category',
 'size',
 'asin',
 'courier_status',
 'qty',
 'currency',
 'amount',
 'ship_city',
 'ship_state',
 'ship_postal_code',
 'ship_country',
 'promotion_ids',
 'b2b',
 'fulfilled_by']

## 3) Convert data types

In [6]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
df['qty'] = pd.to_numeric(df['qty'], errors='coerce')

df[['date','amount','qty']].dtypes


C:\Users\Adil\AppData\Local\Temp\ipykernel_15796\2000358112.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


date      datetime64[ns]
amount           float64
qty                int64
dtype: object

## 4) Flags & simple features

In [7]:
df['has_promo'] = df['promotion_ids'].notna() & (df['promotion_ids'].astype(str).str.strip() != '')
df['is_cancelled'] = df['status'].astype(str).str.contains('cancel', case=False, na=False)

df['month'] = df['date'].dt.to_period('M').astype(str)
df['weekday'] = df['date'].dt.day_name()

df[['has_promo','is_cancelled','month','weekday']].head()


,has_promo,is_cancelled,month,weekday
0,False,True,2022-04,Saturday
1,True,False,2022-04,Saturday
2,True,False,2022-04,Saturday
3,False,True,2022-04,Saturday
4,False,False,2022-04,Saturday


## 5) Data quality quick checks

In [8]:
(df.isna().mean()*100).round(2).sort_values(ascending=False).head(15)

fulfilled_by        69.55
promotion_ids       38.11
currency             6.04
amount               6.04
courier_status       5.33
ship_country         0.03
ship_postal_code     0.03
ship_state           0.03
ship_city            0.03
index                0.00
month                0.00
is_cancelled         0.00
has_promo            0.00
b2b                  0.00
qty                  0.00
dtype: float64

In [9]:
df[(df['qty']<=0) | (df['amount']<0)].shape

(12807, 27)

## 6) Export cleaned dataset

In [10]:
clean_path = 'amazon_sales_clean.csv'
df.to_csv(clean_path, index=False)
clean_path

'amazon_sales_clean.csv'